In [1]:
import pandas as pd
import numpy as np
import cv2 as cv
from PIL import Image,ImageDraw
# import openslide
from skimage import draw
import xml.etree.ElementTree as ET
import os

In [2]:
# list all dirs then get all xml files
# load xml files name in dataframe
root_path = './'
dirs = sorted(os.listdir(root_path + 'data'))

xml_df = pd.DataFrame(columns=['xml'])

dir_names = []
roi_arr = []
for dir in dirs:
    xmls = []
    images = []
    roi_num = 0
    for xml in os.listdir(root_path + 'data/' + dir + '/xml'):
        dir_names.append(dir)
        roi_arr.append(roi_num)
        xml_df.loc[len(xml_df)] = root_path + 'data/' + dir + '/xml/' + xml
        roi_num += 1

xml_df['dir'] = dir_names
xml_df['roi_num'] = roi_arr
xml_df

,xml,dir,roi_num
0,./data/01/xml/1_epidermis_69409_18364_key.xml,01,0
1,./data/01/xml/2_epidermis_69388_10580_key.xml,01,1
2,./data/01/xml/3_epidermis_49235_9209_key.xml,01,2
3,./data/01/xml/4_epidermis_9040_29677_key.xml,01,3
4,./data/01/xml/5_epidermis_10728_47169_key.xml,01,4
...,...,...,...
102,./data/48/xml/1_epidermis_23415_19629_key.xml,48,0
103,./data/48/xml/2_epidermis_104266_47510_key.xml,48,1
104,./data/48/xml/3_epidermis_104903_33862_key.xml,48,2
105,./data/48/xml/4_epidermis_76526_19314_key.xml,48,3


In [7]:
j = 0  # Initialize a counter

# Check if the './masks' directory exists; if not, create it
if not os.path.exists('./masks'):
    os.mkdir('./masks')

# Iterate over each XML file in xml_df['xml']
for xml in xml_df['xml']:
    tree = ET.parse(xml)  # Parse the XML file
    dims = tree.getroot()[0][0].text.split(',')  # Extract image dimensions from XML
    overlay = tree.getroot()[0][1]  # Extract overlay information from XML

    width = int(dims[0])  # Extract width from dimensions
    height = int(dims[1])  # Extract height from dimensions

    mask = np.zeros((height, width)).astype(np.uint8)  # Create an empty mask

    for item in overlay:
        # Extract the list of points defining the region
        point_list = item[-1]
        coords = np.zeros((len(point_list), 2))

        # Extract the coordinates of each point and store them in 'coords' array
        for i, point in enumerate(point_list):
            coords[i][0] = float(point.text.split(',')[0])
            coords[i][1] = float(point.text.split(',')[1])

        # Create a Pillow Image object for drawing the polygon
        polygon = Image.new('1', (width, height))
        draw = ImageDraw.Draw(polygon)

        # Draw the polygon on the image using the coordinates
        draw.polygon(coords.flatten().tolist(), fill=1)
        region_mask = np.array(polygon)

        # Combine the region mask with the overall mask
        mask = np.logical_or(mask, region_mask).astype(np.uint8) * 255

    # Save the mask as a TIFF file
    filename = f'./masks/{xml_df["dir"][j]}_{xml_df["roi_num"][j]}.tif'
    cv.imwrite(filename, mask)

    print(f'{filename} ---------------> Done')

    j += 1  # Increment the counter

./masks/01_0.tif ---------------> Done
./masks/01_1.tif ---------------> Done
./masks/01_2.tif ---------------> Done
./masks/01_3.tif ---------------> Done
./masks/01_4.tif ---------------> Done
./masks/01_5.tif ---------------> Done
./masks/02_0.tif ---------------> Done
./masks/02_1.tif ---------------> Done
./masks/02_2.tif ---------------> Done
./masks/03_0.tif ---------------> Done
./masks/03_1.tif ---------------> Done
./masks/03_2.tif ---------------> Done
./masks/03_3.tif ---------------> Done
./masks/04_0.tif ---------------> Done
./masks/04_1.tif ---------------> Done
./masks/05_0.tif ---------------> Done
./masks/06_0.tif ---------------> Done
./masks/06_1.tif ---------------> Done
./masks/07_0.tif ---------------> Done
./masks/07_1.tif ---------------> Done
./masks/07_2.tif ---------------> Done
./masks/08_0.tif ---------------> Done
./masks/08_1.tif ---------------> Done
./masks/08_2.tif ---------------> Done
./masks/08_3.tif ---------------> Done
./masks/09_0.tif --------